### Importing Required Libraries

In [8]:
import pandas as pd
import datetime as dt 

### Read the 'Seattle Paid Parking Occcupany' Last 48 hrs Dataset into a Pandas Dataframe

In [80]:
df=pd.read_csv("C:\Step 3 - DataCollection\PaidParkingOccupancy.csv")

C:\Users\yogit\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Getting the first 3 records of the Dataframe

In [81]:
df.head(3)

,OccupancyDateTime,PaidOccupancy,BlockfaceName,SideOfStreet,SourceElementKey,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,PaidParkingRate,ParkingCategory,Location
0,11/24/2020 02:33:00 PM,2,S WELLER ST BETWEEN MAYNARD AVE S AND MAYNARD ...,N,"66,617",120,4,Chinatown/ID,Core,NaN,Paid Parking,POINT (-122.32469977 47.59758631)
1,11/24/2020 03:20:00 PM,7,WESTERN AVE BETWEEN VIRGINIA ST AND LENORA ST,NE,"58,806",120,6,Belltown,South,NaN,Paid Parking,POINT (-122.34403853 47.61082404)
2,11/24/2020 03:17:00 PM,1,E PINE ST BETWEEN CRAWFORD PL AND SUMMIT AVE,S,"37,174",120,3,Pike-Pine,NaN,NaN,Paid Parking,POINT (-122.32587891 47.61514809)


In [82]:
df.dtypes

OccupancyDateTime            object
PaidOccupancy                 int64
BlockfaceName                object
SideOfStreet                 object
SourceElementKey             object
ParkingTimeLimitCategory     object
ParkingSpaceCount             int64
PaidParkingArea              object
PaidParkingSubArea           object
PaidParkingRate             float64
ParkingCategory              object
Location                     object
dtype: object

## Schema needs to be changed as follows

|Column|DataType| Desciption|
|-----|-------|-------------|
|OccupancyDateTime|Timestamp|The date and time (minute) of the transaction as recorded|
|PaidOccupancy|INT|Number of transactions that have paid at this date/time on this blockface|
|BlockfaceName|STRING|Street segment, name of street with the “from street” and “to street|
|SideOfStreet|STRING|Options are: E, S, N, W, NE, SW, SE, NW.|
|Station_ID|INT|Unique identifier for the city street segment where the pay station is located;|
|ParkingTimeLimitCategory|INT|In minutes. O|ptions are 120 (2-hour parking), 240 (4-hour parking), 30, or 600 (10-hour parking)
|ParkingSpaceCount|INT|Number of paid spaces on the blockface at the given date and time.|
|PaidParkingArea|STRING|The primary name of a paid parking neighborhood. Example is Commercial Core.|
|PaidParkingSubArea|STRING|A subset of a paid parking area—not all paid parking areas have subareas.|
|PaidParkingRate|FLOAT|Parking rate charged at date and time.|
|ParkingCategory|STRING|An overall description of the type of parking allowed on a blockface.|
|Location|STRING|Calculated based on the known location of a pay station along the same blockface.|

## Rename the column value

In [83]:
df.rename(columns = {'SourceElementKey':'Station_ID'}, inplace = True) 

In [84]:
df.dtypes

OccupancyDateTime            object
PaidOccupancy                 int64
BlockfaceName                object
SideOfStreet                 object
Station_ID                   object
ParkingTimeLimitCategory     object
ParkingSpaceCount             int64
PaidParkingArea              object
PaidParkingSubArea           object
PaidParkingRate             float64
ParkingCategory              object
Location                     object
dtype: object

### Data Cleaning and Adding Additional Columns in Seattle Parking Occupancy Data
<ol>
<li>Need to modify the timestamp column and derive two additional columns for month and day of the week using Pyspark to get the daily and monthly trends</li>
<li>Remove the comma from the source element id</li>
</ol>


### Blockface Data

### Displays blockfaces for all segments of the street network. Identifies the elements of the block, such as peak hour restrictions, length of the block, parking categories, and restricted parking zones.  


In [85]:
blockface=pd.read_csv("C:\SpringBoard-DE-Projects\Step 4 - Data Exploration\Blockface.csv")

C:\Users\yogit\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (34,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [86]:
blockface.head(5)

,OBJECTID,ELMNTKEY,SEGKEY,UNITID,UNITID2,UNITDESC,SIDE,BLOCK_ID,BLOCK_NBR,CSM,...,SUBAREA,START_TIME_WKD,END_TIME_WKD,START_TIME_SAT,END_TIME_SAT,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,OVERRIDECOMMENT,SHAPE_Length
0,6,122148,286965,6487,420,DUWAMISH RIVER TRL BETWEEN WEST MARGINAL WAY S...,W,NaN,4200,N,...,NaN,NaN,NaN,NaN,NaN,DISTRICT1,,N,NaN,1143.579231
1,9,137002,726976,9778,100,SR99 TUNNEL NB BETWEEN ALASKAN WY VI NB AND SR...,W,NaN,0,N,...,NaN,NaN,NaN,NaN,NaN,DISTRICT2,,N,NaN,10528.973330
2,11,136762,724668,4448,40,6TH AV N SR99 ON RP BETWEEN 6TH AVE N AND SR99...,W,NaN,0,N,...,NaN,NaN,NaN,NaN,NaN,DISTRICT7,,N,NaN,295.348033
3,14,136021,692463,1365,1330,23RD AVE NE BETWEEN NE 133RD ST AND NE 135TH W ST,W,NaN,0,N,...,NaN,NaN,NaN,NaN,NaN,DISTRICT5,,N,NaN,634.614692
4,16,69725,1815,405,340,13TH AVE SW BETWEEN 11TH AV SW ACCESS RD AND K...,W,NaN,3400,N,...,NaN,NaN,NaN,NaN,NaN,DISTRICT1,,N,NaN,247.884384


In [90]:
blockface.dtypes

ELMNTKEY                int64
UNITDESC               object
SIDE                   object
BLOCK_NBR               int64
PARKING_CATEGORY       object
WKD_RATE1             float64
WKD_START1              int64
WKD_END1                int64
WKD_RATE2             float64
WKD_START2              int64
WKD_END2                int64
WKD_RATE3             float64
WKD_START3            float64
WKD_END3              float64
SAT_RATE1             float64
SAT_START1            float64
SAT_END1              float64
SAT_RATE2             float64
SAT_START2            float64
SAT_END2              float64
SAT_RATE3             float64
SAT_START3            float64
SAT_END3              float64
PARKING_TIME_LIMIT    float64
SUBAREA                object
dtype: object

## Schema of BlockFace Data needs to be changed as follows

### Drop unwanted columns

In [87]:
blockface = blockface.drop(["OBJECTID","SEGKEY",
                               "UNITID",
                                "UNITID2",
                                "BLOCK_ID",
                                "CSM",
                                "LOAD",
                                "ZONE",
                                "TOTAL_ZONES",
                                "RPZ_ZONE",
                                "RPZ_AREA",
                                "PAIDAREA",
                                "START_TIME_WKD",
                                "END_TIME_WKD",
                                "START_TIME_SAT",
                                "END_TIME_SAT",
                                "PRIMARYDISTRICTCD",
                                "SECONDARYDISTRICTCD",
                                "OVERRIDEYN",
                                "OVERRIDECOMMENT",
                                "SHAPE_Length"],axis=1)

In [88]:
blockface.head(3)

,ELMNTKEY,UNITDESC,SIDE,BLOCK_NBR,PARKING_CATEGORY,WKD_RATE1,WKD_START1,WKD_END1,WKD_RATE2,WKD_START2,...,SAT_START1,SAT_END1,SAT_RATE2,SAT_START2,SAT_END2,SAT_RATE3,SAT_START3,SAT_END3,PARKING_TIME_LIMIT,SUBAREA
0,122148,DUWAMISH RIVER TRL BETWEEN WEST MARGINAL WAY S...,W,4200,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,137002,SR99 TUNNEL NB BETWEEN ALASKAN WY VI NB AND SR...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,136762,6TH AV N SR99 ON RP BETWEEN 6TH AVE N AND SR99...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [89]:
blockface.dtypes

ELMNTKEY                int64
UNITDESC               object
SIDE                   object
BLOCK_NBR               int64
PARKING_CATEGORY       object
WKD_RATE1             float64
WKD_START1              int64
WKD_END1                int64
WKD_RATE2             float64
WKD_START2              int64
WKD_END2                int64
WKD_RATE3             float64
WKD_START3            float64
WKD_END3              float64
SAT_RATE1             float64
SAT_START1            float64
SAT_END1              float64
SAT_RATE2             float64
SAT_START2            float64
SAT_END2              float64
SAT_RATE3             float64
SAT_START3            float64
SAT_END3              float64
PARKING_TIME_LIMIT    float64
SUBAREA                object
dtype: object

In [76]:
blockface.rename(columns={'ELMNTKEY':'Station_ID','UNITDESC':'UnitDesc','SIDE':'Side','BLOCK_NBR':'Block_Nbr','PARKING_CATEGORY':'Parking_Category',
                  'WKD_RATE1':'Wkd_Rate1','WKD_START1':'Wkd_Start1','WKD_END1':'Wkd_End1','WKD_RATE2':'Wkd_Rate2',
                   'WKD_START2':'Wkd_Start2','WKD_END2':'Wkd_End2','WKD_RATE3':'Wkd_Rate3','WKD_START3':'Wkd_Start3',
                   'WKD_END3':'Wkd_End3','SAT_RATE1':'Sat_Rate1','SAT_START1':'Sat_Start1','SAT_END1':'Sat_End1',
                    'SAT_RATE2':'Sat_Rate2','SAT_START2':'Sat_Start2','SAT_END2':'Sat_End2','SAT_RATE3':'Sat_Rate3',
                     'SAT_START3':'Sat_Start3','SAT_END3':'Sat_End3','PARKING_TIME_LIMIT':'Parking_Time_Limit',
                     'SUBAREA':'SubArea'},inplace=True)

In [77]:
blockface.head(3)

,Station_ID,UnitDesc,Side,Block_Nbr,Parking_Category,Wkd_Rate1,Wkd_Start1,Wkd_End1,Wkd_Rate2,Wkd_Start2,...,Sat_Start1,Sat_End1,Sat_Rate2,Sat_Start2,Sat_End2,Sat_Rate3,Sat_Start3,Sat_End3,Parking_Time_Limit,SubArea
0,122148,DUWAMISH RIVER TRL BETWEEN WEST MARGINAL WAY S...,W,4200,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,137002,SR99 TUNNEL NB BETWEEN ALASKAN WY VI NB AND SR...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,136762,6TH AV N SR99 ON RP BETWEEN 6TH AVE N AND SR99...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [78]:
blockface.dtypes

Station_ID              int64
UnitDesc               object
Side                   object
Block_Nbr               int64
Parking_Category       object
Wkd_Rate1             float64
Wkd_Start1              int64
Wkd_End1                int64
Wkd_Rate2             float64
Wkd_Start2              int64
Wkd_End2                int64
Wkd_Rate3             float64
Wkd_Start3            float64
Wkd_End3              float64
Sat_Rate1             float64
Sat_Start1            float64
Sat_End1              float64
Sat_Rate2             float64
Sat_Start2            float64
Sat_End2              float64
Sat_Rate3             float64
Sat_Start3            float64
Sat_End3              float64
Parking_Time_Limit    float64
SubArea                object
dtype: object

### Schema to be changed as follows


|Column|DataType| 
|-----|-------|
|Station_id| INT| 
|UnitDesc|STRING|
|Side| STRING| 
|Block_Nbr|INT|
|Parking_Category| STRING| 
|Wkd_Wate1| DOUBLE| 
|Wkd_Start1| STRING| 
|Wkd_End1| STRING| 
|Wkd_Rate2| DOUBLE| 
|Wkd_Start2| STRING| 
|Wkd_End2| STRING| 
|Wkd_Rate3| DOUBLE| 
|Wkd_Start3| STRING| 
|Wkd_End3| STRING| 
|Sat_Rate1| DOUBLE| 
|Sat_Start1| STRING| 
|Sat_End1| STRING| 
|Sat_Rate2| DOUBLE| 
|Sat_Start2| STRING| 
|Sat_End2| STRING| 
|Sat_Rate3| DOUBLE| 
|Sat_Start3| STRING| 
|Sat_End3| STRING| 
|Parking_Time_Limit| DOUBLE| 
|SubArea| STRING| 

### Paid Parking Transaction Data

In [29]:
paid_parking=pd.read_csv("C:\SpringBoard-DE-Projects\Step 4 - Data Exploration\Paid_Parking_Transaction_Data.csv")

In [28]:
paid_parking.head(5)

,Transaction ID,Meter Code,Transaction DateTime,Payment Mean,Amount Paid,Duration In Minutes,Blockface Name,Side Of Street,Element key,Parking Space Number,Latitude,Longitude
0,910452505,4501002,11/25/2020 04:52:00 AM,CREDIT CARD,1.0,120,MADISON ST BETWEEN 3RD AVE AND 4TH AVE,SE,34214,NaN,47.605808,-122.333418
1,910452607,10255004,11/25/2020 04:53:00 AM,CREDIT CARD,2.0,240,4TH AVE BETWEEN VINE ST AND CEDAR ST,NE,6198,NaN,47.617572,-122.347464
2,910452836,5510002,11/25/2020 04:55:00 AM,CREDIT CARD,1.0,120,1ST AVE BETWEEN SENECA ST AND UNIVERSITY ST,SW,1013,NaN,47.606424,-122.337983
3,910452996,2020002,11/25/2020 04:56:00 AM,CREDIT CARD,1.0,120,WESTERN AVE BETWEEN SPRING ST AND SENECA ST,SW,58801,NaN,47.605281,-122.338434
4,910452997,4081002,11/25/2020 04:56:00 AM,CASH,1.0,120,COLUMBIA ST BETWEEN 4TH AVE AND 5TH AVE,SE,9770,NaN,47.604845,-122.331004


In [12]:
paid_parking.dtypes

Transaction ID            int64
Meter Code                int64
Transaction DateTime     object
Payment Mean             object
Amount Paid             float64
Duration In Minutes       int64
Blockface Name           object
Side Of Street           object
Element key               int64
Parking Space Number    float64
Latitude                float64
Longitude               float64
dtype: object

## Schema needs to be changed as follows

                                    
|Column|DataType| 
|-----|-------|
|Transaction ID|INT|
|Meter Code|INT|
|Transaction DateTime|TIMESTAMP|
|Payment Mean|STRING|
|Amount Paid|FLOAT|
|Duration In Minutes|INT|
|Blockface Name |STRING|
|Side Of Street|STRING|
|Station_ID|INT|
|Parking Space Number |FLOAT|
|Latitude|FLOAT|
|Longitude|FLOAT|